In [1]:
# -*- coding: utf-8 -*-

import numpy as np
import os
import cv2
import sys
from PyQt5.QtWidgets import (QApplication, QWidget,
                             QFileDialog,
                             QSlider, QPushButton, QLabel,
                             QHBoxLayout, QVBoxLayout)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import (QPixmap, QIcon)



class Sampling_Creator_Main_Class(QWidget):
    
    
    def __init__(self):
        super().__init__()
        
        self.path_to_pull = None
        self.list_dir = []
        self.list_dir_iterator = 0
        
        self.sld_H = QSlider(Qt.Vertical, self)
        self.sld_L = QSlider(Qt.Vertical, self)
        self.sld_S = QSlider(Qt.Vertical, self)
        self.sld_A = QSlider(Qt.Vertical, self)
        
        self.picture = QLabel(self)
        
        self.btn_previous = QPushButton('Previous', self)
        self.btn_next = QPushButton('Next', self)
        self.btn_save = QPushButton('Save', self)
        
        self.sld_box = QHBoxLayout(self)
        self.btn_box = QHBoxLayout(self)
        self.picture_box = QVBoxLayout(self)
        self.picture_and_btn_box = QVBoxLayout(self)
        self.all_box = QHBoxLayout(self)
        
        self.tmp = None
        self.tmp_im_non_rotated = None
        
        self.old_value_H = 0
        self.old_value_L = 0
        self.old_value_S = 0

        self.btn_save_was_clicked = False
        self.btn_next_or_prev_was_clicked = False
        self.path_to_push = None
        
        self.initUI()

        
    def initUI(self):        
        self.path_to_pull = str(QFileDialog.getExistingDirectory(self, 'Select Directory'))
        self.list_dir = os.listdir(self.path_to_pull)
        self.tmp_im = cv2.imread(self.path_to_pull + '/' + self.list_dir[self.list_dir_iterator])
        self.tmp_im_non_rotated = self.tmp_im.copy()

        self.create_sliders()
        self.create_picture()
        self.create_buttons()
        self.create_box()
        
        self.setWindowTitle('Sampling Creator')
        self.setWindowIcon(QIcon('/common/icon.png'))
        self.showMaximized()

        
    def create_sliders(self):
        self.sld_H.setMinimum(-40)
        self.sld_H.setMaximum(40)
        self.sld_H.setValue(0)
        self.sld_H.valueChanged[int].connect(self.changeValue_H)
        
        self.sld_L.setMinimum(-40)
        self.sld_L.setMaximum(40)
        self.sld_L.setValue(0)
        self.sld_L.valueChanged[int].connect(self.changeValue_L)
        
        self.sld_S.setMinimum(-40)
        self.sld_S.setMaximum(40)
        self.sld_S.setValue(0)
        self.sld_S.valueChanged[int].connect(self.changeValue_S)
        
        self.sld_A.setMinimum(-180)
        self.sld_A.setMaximum(180)
        self.sld_A.setValue(0)
        self.sld_A.valueChanged[int].connect(self.changeValue_A)
        
        
    def create_picture(self):
        self.picture.setPixmap(QPixmap(self.path_to_pull + '/' + self.list_dir[self.list_dir_iterator]))
        self.picture.setScaledContents(True)
        self.picture.resize(300, 300)
        
        
    def create_buttons(self):
        self.btn_previous.clicked.connect(self.buttonClicked)
        self.btn_next.clicked.connect(self.buttonClicked)
        self.btn_save.clicked.connect(self.buttonClicked)
        
        
    def create_box(self):
        self.sld_box.addWidget(self.sld_H)
        self.sld_box.addWidget(self.sld_L)
        self.sld_box.addWidget(self.sld_S)
        self.sld_box.addWidget(self.sld_A)
        
        self.btn_box.addWidget(self.btn_previous)
        self.btn_box.addWidget(self.btn_next)
        self.btn_box.addWidget(self.btn_save)
        
        self.picture_box.addWidget(self.picture)
        
        self.picture_and_btn_box.addWidget(self.picture_box)
        self.picture_and_btn_box.addWidget(self.btn_box)
        
        self.all_box.addWidget(self.sld_box)
        self.all_box.addLayout(self.picture_and_btn_box)
        
        self.setLayout(self.all_box)
        
        
    def changeValue_H(self, value):
        if (self.btn_next_or_prev_was_clicked == False):
            hls_im = cv2.cvtColor(self.tmp_im, cv2.COLOR_RGB2HLS)
            hls_im[:, :, 0] = hls_im[:, :, 0] + value - self.old_value_H
            self.tmp_im = cv2.cvtColor(hls_im, cv2.COLOR_HLS2RGB)
        
            hls_im = cv2.cvtColor(self.tmp_im_non_rotated, cv2.COLOR_RGB2HLS)
            hls_im[:, :, 0] = hls_im[:, :, 0] + value - self.old_value_H
            self.tmp_im_non_rotated = cv2.cvtColor(hls_im, cv2.COLOR_HLS2RGB)
        
            self.old_value_H = value
        
            cv2.imwrite('common/tmp_im.jpg', self.tmp_im)
            self.picture.setPixmap(QPixmap('common/tmp_im.jpg'))
        
        
    def changeValue_L(self, value):
        if (self.btn_next_or_prev_was_clicked == False):
            hls_im = cv2.cvtColor(self.tmp_im, cv2.COLOR_RGB2HLS)
            hls_im[:, :, 1] = hls_im[:, :, 1] + value - self.old_value_L
            self.tmp_im = cv2.cvtColor(hls_im, cv2.COLOR_HLS2RGB)
        
            hls_im = cv2.cvtColor(self.tmp_im_non_rotated, cv2.COLOR_RGB2HLS)
            hls_im[:, :, 1] = hls_im[:, :, 1] + value - self.old_value_L
            self.tmp_im_non_rotated = cv2.cvtColor(hls_im, cv2.COLOR_HLS2RGB)
        
            self.old_value_L = value
        
            cv2.imwrite('common/tmp_im.jpg', self.tmp_im)
            self.picture.setPixmap(QPixmap('common/tmp_im.jpg'))
        
        
    def changeValue_S(self, value):
        if (self.btn_next_or_prev_was_clicked == False):
            hls_im = cv2.cvtColor(self.tmp_im, cv2.COLOR_RGB2HLS)
            hls_im[:, :, 2] = hls_im[:, :, 2] + value - self.old_value_S
            self.tmp_im = cv2.cvtColor(hls_im, cv2.COLOR_HLS2RGB)
        
            hls_im = cv2.cvtColor(self.tmp_im_non_rotated, cv2.COLOR_RGB2HLS)
            hls_im[:, :, 2] = hls_im[:, :, 2] + value - self.old_value_S
            self.tmp_im_non_rotated = cv2.cvtColor(hls_im, cv2.COLOR_HLS2RGB)
        
            self.old_value_S = value
        
            cv2.imwrite('common/tmp_im.jpg', self.tmp_im)
            self.picture.setPixmap(QPixmap('common/tmp_im.jpg'))
        
        
    def changeValue_A(self, value):
        if (self.btn_next_or_prev_was_clicked == False):
            self.tmp_im = self.rotate_bound(self.tmp_im_non_rotated, value)
        
            cv2.imwrite('common/tmp_im.jpg', self.tmp_im)
            self.picture.setPixmap(QPixmap('common/tmp_im.jpg'))
        
        
    def buttonClicked(self):
        sender = self.sender()
        if (sender.text() == 'Next' and self.list_dir_iterator < len(self.list_dir) - 1):
            self.list_dir_iterator += 1
            self.btn_next_or_prev_was_clicked = True
            self.sld_H.setValue(0)
            self.sld_L.setValue(0)
            self.sld_S.setValue(0)
            self.sld_A.setValue(0)
            
            self.tmp_im = cv2.imread(self.path_to_pull + '/' + self.list_dir[self.list_dir_iterator])
            self.tmp_im_non_rotated = self.tmp_im.copy()
            self.picture.setPixmap(QPixmap(self.path_to_pull + '/' + self.list_dir[self.list_dir_iterator]))
            
            self.btn_next_or_prev_was_clicked = False
            
        elif (sender.text() == 'Previous' and self.list_dir_iterator > 0):
            self.list_dir_iterator -= 1
            self.btn_next_or_prev_was_clicked = True
            self.sld_H.setValue(0)
            self.sld_L.setValue(0)
            self.sld_S.setValue(0)
            self.sld_A.setValue(0)
            
            self.tmp_im = cv2.imread(self.path_to_pull + '/' + self.list_dir[self.list_dir_iterator])
            self.tmp_im_non_rotated = self.tmp_im.copy()
            self.picture.setPixmap(QPixmap(self.path_to_pull + '/' + self.list_dir[self.list_dir_iterator]))
            
            self.btn_next_or_prev_was_clicked = False
            
        elif (sender.text() == 'Save'):
            if (self.btn_save_was_clicked == False):
                self.btn_save_was_clicked = True
                self.path_to_push = str(QFileDialog.getExistingDirectory(self, 'Select Directory'))

            cv2.imwrite(self.path_to_push + '/' + self.list_dir[self.list_dir_iterator], self.tmp_im)
        
        
    def rotate_bound(self, image, angle):
        # grab the dimensions of the image and then determine the
        # center
        (h, w) = image.shape[:2]
        (cX, cY) = (w // 2, h // 2) 
        # grab the rotation matrix (applying the negative of the
        # angle to rotate clockwise), then grab the sine and cosine
        # (i.e., the rotation components of the matrix)
        M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1]) 
        # compute the new bounding dimensions of the image
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin)) 
        # adjust the rotation matrix to take into account translation
        M[0, 2] += (nW / 2) - cX
        M[1, 2] += (nH / 2) - cY
        # perform the actual rotation and return the image
        return cv2.warpAffine(image, M, (nW, nH))

    

if __name__ == '__main__':
    Sampling_Creator = QApplication(sys.argv)
    Sampling_Creator_Instance = Sampling_Creator_Main_Class()
    sys.exit(Sampling_Creator.exec_())

TypeError: addWidget(self, QWidget, stretch: int = 0, alignment: Union[Qt.Alignment, Qt.AlignmentFlag] = 0): argument 1 has unexpected type 'QVBoxLayout'